<a href="https://colab.research.google.com/github/merryljohns/breast-cancer-prediction/blob/main/SalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%pip install -Uq upgini catboost

In [25]:
from os.path import exists
import pandas as pd
df_path="train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df=pd.read_csv(df_path)
df=df.sample(n=19_000,random_state=0)
df["store"]=df["store"].astype(str)
df["item"]=df["item"].astype(str)
df["date"]=pd.to_datetime(df["date"])
df.sort_values("date",inplace=True)
df.reset_index(inplace=True,drop=True)
df.head()
print(df.shape)

(19000, 4)


In [26]:
train =df[df["date"]<"2017-01-01"]
test=df[df["date"]>="2017-01-01"]

In [27]:
train_features=train.drop(columns=["sales"])
train_target=train["sales"]
test_features=train.drop(columns=["sales"])
test_target=train["sales"]



In [28]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType
enricher=FeaturesEnricher(
    search_keys={
        "date":SearchKey.DATE
    },
    cv=CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features,test_target)])

[============================================================] 100% Finished

Eval set 1 is equal to train set and will be ignored
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
target,All valid,-
date,All valid,-




Running search request, search_id=663575c3-7b4c-41ec-bd5e-78e1374cc026
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com



/usr/local/lib/python3.12/dist-packages/upgini/features_enricher.py:4491: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if self.features_info[self.bundle.get("features_info_psi")].fillna(0.0).eq(0.0).all():
/usr/local/lib/python3.12/dist-packages/upgini/features_enricher.py:4491: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if self.features_info[self.bundle.get("features_info_psi")].fillna(0.0).eq(0.0).all():


...

Training dataset,AutoFE: features from Training dataset,13.0295,3
Upgini,Calendar data,7.9743,4
"Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",3.3418,2
Upgini,Markets data,2.1846,1
Upgini,World economic indicators,0.2487,1


Training dataset,f_autofe_lag_28d_aaadd46bbb,target,lag_28d
"Training dataset,Calendar data",f_autofe_roll_3d_min_1a2ad6d8bb,f_events_date_year_cos1_9014a856,roll_3d_min
"Training dataset,Calendar data",f_autofe_roll_3d_median_8e12f1e1e9,f_events_date_year_cos1_9014a856,roll_3d_median
Training dataset,f_autofe_roll_3d_median_97804b26e6,target,roll_3d_median
Training dataset,f_autofe_lag_7d_4a1d9c9dff,target,lag_7d


We detected 25 outliers in your sample.
Examples of outliers with maximum value of target:
17    196
12    187
15    171
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...


/usr/local/lib/python3.12/dist-packages/upgini/features_enricher.py:4491: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if self.features_info[self.bundle.get("features_info_psi")].fillna(0.0).eq(0.0).all():


Train,9418,53.3352,0.324 ± 0.109,0.299 ± 0.100,0.0250,7.7%


In [29]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric
model=CatBoostRegressor(verbose=False,allow_writing_files=False,random_state=0)
enricher.calculate_metrics(
    train_features,train_target,
    eval_set=[(test_features,test_target)],
    estimator=model,
    scoring="mean_absolute_percentage_error"
)

Eval set 1 is equal to train set and will be ignored
Eval set 1 is equal to train set and will be ignored
Calculating accuracy uplift after enrichment...


/usr/local/lib/python3.12/dist-packages/upgini/features_enricher.py:4491: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if self.features_info[self.bundle.get("features_info_psi")].fillna(0.0).eq(0.0).all():


,Dataset type,Rows,Mean target,Baseline MAPE,Enriched MAPE,"Uplift, abs","Uplift, %"
0,Train,9418,53.3352,0.288 ± 0.096,0.229 ± 0.112,0.059,20.5%


In [31]:
sampled_train_features = train_features.sample(n=1000, random_state=42)
sampled_train_target = train_target.loc[sampled_train_features.index]

enriched_train_features = enricher.transform(
    sampled_train_features,
    y=sampled_train_target,
    keep_input=True
)

enriched_train_features.head()


[============================================================] 100% Finished

You are trying to launch enrichment for 1000 rows, which will exceed the rest limit 148.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
model.fit(train_features,train_target)
preds=model.predict(test_features)
eval_metric(test_target.values,preds,'SMAPE')

In [23]:
# Split within trial limit
train_sample = 800
test_sample = 200

# Sample train and test
sampled_train_features = train_features.sample(n=train_sample, random_state=42)
sampled_train_target = train_target.loc[sampled_train_features.index]

sampled_test_features = test_features.sample(n=test_sample, random_state=42)
sampled_test_target = test_target.loc[sampled_test_features.index]

# Enrich train
enriched_train_features = enricher.transform(
    sampled_train_features,
    y=sampled_train_target,
    keep_input=True
)

# Enrich test (add y here to avoid ValidationError)
enriched_test_features = enricher.transform(
    sampled_test_features,
    y=sampled_test_target,
    keep_input=True
)

# Train model
model.fit(enriched_train_features, sampled_train_target)

# Predict and evaluate
enriched_preds = model.predict(enriched_test_features)
score = eval_metric(sampled_test_target.values, enriched_preds, 'SMAPE')
print(f"SMAPE: {score}")


[                                                            ] 0% Checking dataset...

You are trying to launch enrichment for 800 rows, which will exceed the rest limit 321.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 679 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

WARNING #2: 13.0653% of rows (26) in X and eval_set are duplicates with different y values. These rows will be deleted as incorrect
Sample of incorrect row indexes: [4, 15, 16, 22, 32, 34, 35, 65, 75, 81, 83, 100, 107, 109, 111, 114, 115, 117, 119, 121, 125, 151, 153, 160, 161, 168]



Column name,Status,Errors
date,All valid,-




Running transform request, id=d50e2d27-d766-4e18-95f1-2c1514a97e07
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


CatBoostError: X must not be None